<a href="https://colab.research.google.com/github/vhrique/aerogerador_fault_diagnosis/blob/main/baseline_experiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.preprocessing import OrdinalEncoder

In [17]:
df_features = pd.read_csv('/content/drive/MyDrive/Estudos/Datasets/pas_aerogerador_helon/X10.csv', header=None)
df_labels = pd.read_csv('/content/drive/MyDrive/Estudos/Datasets/pas_aerogerador_helon/y_case2.csv', header=None)

In [20]:
X = df_features.to_numpy()
y = df_labels.to_numpy()

In [27]:
y_pred = y.copy()

for i, (train_index, test_index) in enumerate(KFold(n_splits=5, shuffle=True).split(X)):
    X_train, y_train = X[train_index], y[train_index]
    X_test = X[test_index]
    mdl = RandomForestClassifier().fit(X_train, y_train)
    y_pred[test_index,0] = mdl.predict(X_test)

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/usr/local/lib/python3.10/dist-packages/sklearn/base.py:1152: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example usi

In [29]:
print(
    classification_report(
        y, y_pred
    )
)

              precision    recall  f1-score   support

         0.0       1.00      0.99      0.99       143
         1.0       1.00      1.00      1.00       144
         2.0       1.00      1.00      1.00       144
         3.0       0.99      0.97      0.98       144
         4.0       0.96      0.99      0.98       199
         5.0       0.98      0.96      0.97       168
         6.0       0.99      1.00      1.00       162
         7.0       1.00      0.99      1.00       150
         8.0       1.00      1.00      1.00       156
         9.0       1.00      1.00      1.00       155
        10.0       1.00      1.00      1.00       157
        11.0       1.00      1.00      1.00       152
        12.0       0.99      1.00      1.00       528

    accuracy                           0.99      2402
   macro avg       0.99      0.99      0.99      2402
weighted avg       0.99      0.99      0.99      2402

